## Rules
1. Four coloured team
2. Each team must be covered by 1 person everyday
* On weekdays, one employee can cover 2 teams
* On weekends, employees can only be on one team, but can cover 2 teams if in gold
* One cannot do both Red and Blue on weekends
* An employee cannot work for 3 weekends in a row
* Different days of the week have score. More the score, less ideal to work on:

Saturday < Friday < Sunday < Tuesday = Thursday < Wednesday = Monday<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(5)
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(4)
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(3)
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(2)
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(2)
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(1)
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(1)
       
* Design the algorithm to find the best schedule based on the lowest overall total
* Some employees will request days off, they should not be penaltized to have more work

In [1]:
import random
import calendar
import pandas as pd
import numpy as np
from numpy.random import choice
import timeit
import csv
import warnings
warnings.filterwarnings("ignore")

start = timeit.default_timer()

#### Random year & month

In [2]:
year = random.randint(1970,2100)
month = random.randint(1,12)
print("Year :" , year," Month: ",month)

#no of days
start_day, no_of_days = calendar.monthrange(year, month)
start_day = (calendar.day_name[start_day])
# print("No. of days: ", no_of_days)
# print("Starting day of the week: ", start_day)

Year : 2018  Month:  8


#### Create month table

In [3]:
def get_score(weekday_name):
#     print(weekday_name)
    day_score = []
    for name in weekday_name:
        if name == 'Saturday' :
            day_score.append(5)
        elif name == 'Friday':
            day_score.append(4)
        elif name == 'Sunday':
            day_score.append(3)
        elif name == 'Tuesday' or name =='Thursday' :
            day_score.append(2)
        else:
            day_score.append(1)
    return day_score
            

def create_table(start, end):
    df = pd.DataFrame({'Date': pd.date_range(start, end)})
    df['Day'] = df.Date.dt.weekday_name
    df['Score'] = get_score(df.Date.dt.weekday_name)
    df['Blue'] = 'NA'
    df['Red'] = 'NA'
    df['Silver'] = 'NA'
    df['Gold'] = 'NA'
    return df

In [4]:
start_date = str(year) + '-' + str(month) +'-01'
end_date = str(year) + '-' + str(month) +'-'+str(no_of_days)

df_teams = create_table(start_date, end_date)

#### List of random no. of employees
#### Also select last 2 weekend's work randomly 

In [5]:
no_of_empl = random.randint(25,50)

#6 random employees worked 2 weekends back
weekend_1 = random.sample(range(1,no_of_empl+1), 6)
weekend_2 = random.sample(range(1,no_of_empl+1), 6)

empl_list = []
for i in range(1,no_of_empl+1):
    one_weekend_back = 0
    two_weekend_back = 0
    if i in weekend_1:
        one_weekend_back = 1
    if i in weekend_2:
        two_weekend_back = 1
    if(i<10):
        empl_list.append(['emp_0' + str(i), 0, two_weekend_back, one_weekend_back, 0])
    else:
        empl_list.append(['emp_' + str(i), 0, two_weekend_back, one_weekend_back, 0])

#### Random no. of employee on holiday for random amount of days

In [6]:
#5-10 employees taking a leave
no_of_empl_leave = random.randint(5,10)

#selecting random employees who are takeing a leave
empl_list_leave = random.sample(range(0,no_of_empl+1), no_of_empl_leave)

#assiging random amount of leave days (max 10)
for loc,empl in enumerate(empl_list):
    if loc in empl_list_leave:
        empl[4] = random.randint(1,10)

for num,empl in enumerate(empl_list):
    days_leave = random.sample(list(df_teams['Date']), empl[4])
    empl.append(days_leave)
    empl.append(10000)
    
print("[Employee name, If working this week, If worked one weekend back, if worked two weekends back, No of days on leave, Weight]")
for empl in empl_list:
    print(empl)

[Employee name, If working this week, If worked one weekend back, if worked two weekends back, No of days on leave, Weight]
['emp_01', 0, 1, 0, 0, [], 10000]
['emp_02', 0, 0, 1, 0, [], 10000]
['emp_03', 0, 0, 0, 0, [], 10000]
['emp_04', 0, 0, 1, 0, [], 10000]
['emp_05', 0, 0, 1, 0, [], 10000]
['emp_06', 0, 0, 0, 0, [], 10000]
['emp_07', 0, 1, 0, 4, [Timestamp('2018-08-06 00:00:00'), Timestamp('2018-08-28 00:00:00'), Timestamp('2018-08-24 00:00:00'), Timestamp('2018-08-14 00:00:00')], 10000]
['emp_08', 0, 0, 0, 0, [], 10000]
['emp_09', 0, 0, 0, 0, [], 10000]
['emp_10', 0, 1, 0, 0, [], 10000]
['emp_11', 0, 0, 1, 0, [], 10000]
['emp_12', 0, 1, 0, 0, [], 10000]
['emp_13', 0, 1, 0, 7, [Timestamp('2018-08-13 00:00:00'), Timestamp('2018-08-07 00:00:00'), Timestamp('2018-08-22 00:00:00'), Timestamp('2018-08-30 00:00:00'), Timestamp('2018-08-14 00:00:00'), Timestamp('2018-08-11 00:00:00'), Timestamp('2018-08-17 00:00:00')], 10000]
['emp_14', 0, 1, 0, 0, [], 10000]
['emp_15', 0, 0, 0, 9, [Timest

In [7]:
def employee_schedule(start, end, empl_list):
    df = pd.DataFrame({'Date': pd.date_range(start, end)})
    for emp in empl_list:
        df[emp[0]] = 0
    return df
    
df_emp = employee_schedule(start_date, end_date, empl_list)
# df_emp

#### Randomly assigning work to employees

In [8]:
def update_df_emp(teams, empl_choice, loc, score, df_emp):
    
    for empl in empl_list:
        if empl == empl_choice:
            df_emp.loc[loc,empl[0]] += score*teams
                
    return df_emp

def weighted_choice(free_list):
    totals = []
    running_total = 0

    for w in [empl[6] for empl in free_list]:
        running_total += w
        totals.append(running_total)

    rnd = random.random() * running_total
    for i, total in enumerate(totals):
        if rnd < total:
            return free_list[i]

In [9]:
df_emp.loc['Total'] = 0.0001
for loc,date in enumerate(df_teams['Date']):

    score = df_teams.loc[loc,'Score']
    #weekend
    if(date.weekday_name == 'Saturday' or date.weekday_name == 'Sunday'):
        
        free_list = []
        for empl in empl_list:
            if ((empl[2] == 0 or empl[3] == 0) and empl[1] == 0) and (date not in empl[5]):
                free_list.append(empl)
                
        gold_choice = weighted_choice(free_list)
        df_teams.at[loc, 'Gold'] = gold_choice[0]

        free_list = [x for x in free_list if x != gold_choice]
#         weights = weights.drop(gold_choice[0])
#         del weights[int(gold_choice[0].split('_')[1])-1]
    
        #assign same to one of the other three team (red,blue,silver)
        with_gold = choice(['Red','Blue','Silver'])
        df_teams.at[loc, with_gold] = gold_choice[0]
        update_df_emp(2, gold_choice, loc, score, df_emp)

        #assign different employees to remaining 2 team
        other_choice_1 = weighted_choice(free_list)
        free_list = [x for x in free_list if x != other_choice_1]
#         weights = weights.drop(other_choice_1[0])
#         del weights[int(other_choice_1[0].split('_')[1])-1]
        other_choice_2 = weighted_choice(free_list)

        if with_gold == 'Blue':
            df_teams.at[loc, 'Red'] = other_choice_1[0]
            df_teams.at[loc, 'Silver'] = other_choice_2[0]
            update_df_emp(1,other_choice_1,loc, score, df_emp)
            update_df_emp(1,other_choice_2,loc, score, df_emp)
        elif with_gold == 'Red':
            df_teams.at[loc, 'Blue'] = other_choice_1[0]
            df_teams.at[loc, 'Silver'] = other_choice_2[0]
            update_df_emp(1,other_choice_1,loc, score, df_emp)
            update_df_emp(1,other_choice_2,loc, score, df_emp)
        else:
            df_teams.at[loc, 'Blue'] = other_choice_1[0]
            df_teams.at[loc, 'Red'] = other_choice_2[0]
            update_df_emp(1,other_choice_1,loc, score, df_emp)
            update_df_emp(1,other_choice_2,loc, score, df_emp)

        #update weekend work status
        for i in range(len(empl_list)):
            if(empl_list[i]==gold_choice or empl_list[i]==other_choice_1 or empl_list[i]==other_choice_2):
                empl_list[i][1] = 1

        if(date.weekday_name == 'Sunday'):
            for i in range(len(empl_list)):
                empl_list[i][3] = empl_list[i][2]
                empl_list[i][2] = empl_list[i][1]
                empl_list[i][1] = 0
        
        
#   Weekday
    else:
        free_list = []
        for empl in empl_list:
            if date not in empl[5]:
                free_list.append(empl)
#                 weights.append(df_emp.loc['Total'][empl[0]])
        
        #Get random choice for Gold
        gold_choice = weighted_choice(free_list)
        df_teams.at[loc, 'Gold'] = gold_choice[0]

        #assign same to one of the other three team (red,blue,silver)
        with_gold = choice(['Red','Blue','Silver'])
        df_teams.at[loc, with_gold] = gold_choice[0]
#         weights = weights.drop(gold_choice[0])
#         del weights[int(gold_choice[0].split('_')[1])-1]

        
        update_df_emp(2, gold_choice, loc, score, df_emp)
        free_list = [x for x in empl_list if x != gold_choice]
        
        #assign different employee to remaining 2 team
        other_choice = weighted_choice(free_list)
        if with_gold == 'Blue':
            df_teams.at[loc, 'Red'] = other_choice[0]
            df_teams.at[loc, 'Silver'] = other_choice[0]
            update_df_emp(2,other_choice,loc, score, df_emp)
        elif with_gold == 'Red':
            df_teams.at[loc, 'Blue'] = other_choice[0]
            df_teams.at[loc, 'Silver'] = other_choice[0]
            update_df_emp(2,other_choice,loc, score, df_emp)
        else:
            df_teams.at[loc, 'Blue'] = other_choice[0]
            df_teams.at[loc, 'Red'] = other_choice[0]
            update_df_emp(2,other_choice,loc, score, df_emp)
    
    df_emp.loc['Total'] = 0.0001
    df_emp.loc['Total'] = df_emp.sum()
    for empl in empl_list:
        empl[6] = 1 / df_emp.loc['Total'][empl[0]]

In [10]:
df_emp

Date   emp_01   emp_02   emp_03   emp_04  emp_05  \
0      2018-08-01 00:00:00   0.0000   0.0000   0.0000   0.0000  0.0000   
1      2018-08-02 00:00:00   0.0000   0.0000   0.0000   0.0000  0.0000   
2      2018-08-03 00:00:00   8.0000   0.0000   0.0000   0.0000  0.0000   
3      2018-08-04 00:00:00   0.0000   0.0000   0.0000   0.0000  0.0000   
4      2018-08-05 00:00:00   0.0000   0.0000   0.0000   0.0000  3.0000   
5      2018-08-06 00:00:00   0.0000   0.0000   0.0000   0.0000  0.0000   
6      2018-08-07 00:00:00   0.0000   0.0000   0.0000   4.0000  0.0000   
7      2018-08-08 00:00:00   0.0000   0.0000   0.0000   0.0000  0.0000   
8      2018-08-09 00:00:00   0.0000   0.0000   4.0000   0.0000  0.0000   
9      2018-08-10 00:00:00   0.0000   0.0000   0.0000   0.0000  0.0000   
10     2018-08-11 00:00:00   0.0000   5.0000   0.0000   0.0000  0.0000   
11     2018-08-12 00:00:00   0.0000   0.0000   0.0000   0.0000  0.0000   
12     2018-08-13 00:00:00   0.0000   0.0000   0.0000   0.0000  0.0000   
13     2018-08-14 00:00:00   0.0000   0.0000   0.0000   0.0000  0.0000   
14     2018-08-15 00:00:00   0.0000   2.0000   0.0000   0.0000  0.0000   
15     2018-08-16 00:00:00   0.0000   0.0000   0.0000   0.0000  0.0000   
16     2018-08-17 00:00:00   0.0000   0.0000   0.0000   0.0000  0.0000   
17     2018-08-18 00:00:00   0.0000   0.0000   0.0000   0.0000  0.0000   
18     2018-08-19 00:00:00   0.0000   0.0000   0.0000   0.0000  0.0000   
19     2018-08-20 00:00:00   0.0000   0.0000   0.0000   0.0000  0.0000   
20     2018-08-21 00:00:00   0.0000   0.0000   0.0000   0.0000  0.0000   
21     2018-08-22 00:00:00   2.0000   0.0000   0.0000   0.0000  0.0000   
22     2018-08-23 00:00:00   0.0000   0.0000   4.0000   0.0000  0.0000   
23     2018-08-24 00:00:00   0.0000   0.0000   8.0000   0.0000  0.0000   
24     2018-08-25 00:00:00   0.0000   0.0000   0.0000   0.0000  0.0000   
25     2018-08-26 00:00:00   0.0000   0.0000   0.0000   6.0000  0.0000   
26     2018-08-27 00:00:00   0.0000   0.0000   0.0000   0.0000  0.0000   
27     2018-08-28 00:00:00   0.0000   4.0000   0.0000   0.0000  0.0000   
28     2018-08-29 00:00:00   0.0000   0.0000   0.0000   0.0000  2.0000   
29     2018-08-30 00:00:00   0.0000   0.0000   0.0000   0.0000  0.0000   
30     2018-08-31 00:00:00   0.0000   0.0000   0.0000   0.0000  0.0000   
Total                  NaN  10.0001  11.0001  16.0001  10.0001  5.0001   

        emp_06  emp_07  emp_08   emp_09   ...     emp_23   emp_24  emp_25  \
0       0.0000  0.0000  0.0000   0.0000   ...     0.0000   0.0000  0.0000   
1       0.0000  0.0000  0.0000   0.0000   ...     4.0000   0.0000  0.0000   
2       8.0000  0.0000  0.0000   0.0000   ...     0.0000   0.0000  0.0000   
3       0.0000  0.0000  0.0000   0.0000   ...     0.0000   0.0000  5.0000   
4       0.0000  0.0000  0.0000   0.0000   ...     0.0000   0.0000  0.0000   
5       0.0000  0.0000  0.0000   0.0000   ...     0.0000   0.0000  0.0000   
6       0.0000  0.0000  0.0000   0.0000   ...     0.0000   0.0000  0.0000   
7       0.0000  0.0000  0.0000   0.0000   ...     0.0000   0.0000  0.0000   
8       0.0000  0.0000  0.0000   0.0000   ...     0.0000   0.0000  0.0000   
9       0.0000  0.0000  0.0000   0.0000   ...     0.0000   0.0000  0.0000   
10      0.0000  0.0000  0.0000   0.0000   ...     0.0000   5.0000  0.0000   
11      0.0000  0.0000  6.0000   3.0000   ...     0.0000   0.0000  0.0000   
12      0.0000  2.0000  0.0000   0.0000   ...     0.0000   0.0000  0.0000   
13      0.0000  0.0000  0.0000   0.0000   ...     0.0000   0.0000  0.0000   
14      0.0000  0.0000  0.0000   0.0000   ...     0.0000   0.0000  0.0000   
15      0.0000  0.0000  0.0000   0.0000   ...     0.0000   0.0000  0.0000   
16      0.0000  0.0000  0.0000   0.0000   ...     0.0000   0.0000  0.0000   
17      0.0000  5.0000  0.0000   0.0000   ...     0.0000   0.0000  0.0000   
18      0.0000  0.0000  3.0000   0.0000   ...     0.0000   6.0000  0.0000   
19      0.0000  0.0000  0.0000   0.0000

In [11]:
workload_array = df_emp.loc['Total','emp_01':]
for num,workload in enumerate(workload_array):
    workload *= no_of_days / (no_of_days-empl_list[num][4])
    workload_array[num] = workload
wordload_std = np.std(workload_array)
std_no_of_empl = wordload_std * np.sqrt(no_of_empl)
score_to_minimize = wordload_std * np.sqrt(no_of_empl) / np.sqrt(no_of_days)

In [12]:
# tosave = [no_of_empl, no_of_days, wordload_std, std_no_of_empl, score_to_minimize]
# with open('std.csv','a') as f:
#     writer = csv.writer(f)
#     writer.writerow(tosave)

In [13]:
stop = timeit.default_timer()

print('Time: ', stop - start)  

Time:  0.8592019330244511


In [14]:
score_to_minimize

4.65788991463463

In [30]:
# def test(k):
#     k += 1
#     return k

# m=5
# test(m)
# # print(m)
update_df_emp(1,other_choice_1,loc, score, df_emp)

Date   emp_01   emp_02   emp_03   emp_04  emp_05  \
0      2018-08-01 00:00:00   0.0000   0.0000   0.0000   0.0000  0.0000   
1      2018-08-02 00:00:00   0.0000   0.0000   0.0000   0.0000  0.0000   
2      2018-08-03 00:00:00   8.0000   0.0000   0.0000   0.0000  0.0000   
3      2018-08-04 00:00:00   0.0000   0.0000   0.0000   0.0000  0.0000   
4      2018-08-05 00:00:00   0.0000   0.0000   0.0000   0.0000  3.0000   
5      2018-08-06 00:00:00   0.0000   0.0000   0.0000   0.0000  0.0000   
6      2018-08-07 00:00:00   0.0000   0.0000   0.0000   4.0000  0.0000   
7      2018-08-08 00:00:00   0.0000   0.0000   0.0000   0.0000  0.0000   
8      2018-08-09 00:00:00   0.0000   0.0000   4.0000   0.0000  0.0000   
9      2018-08-10 00:00:00   0.0000   0.0000   0.0000   0.0000  0.0000   
10     2018-08-11 00:00:00   0.0000   5.0000   0.0000   0.0000  0.0000   
11     2018-08-12 00:00:00   0.0000   0.0000   0.0000   0.0000  0.0000   
12     2018-08-13 00:00:00   0.0000   0.0000   0.0000   0.0000  0.0000   
13     2018-08-14 00:00:00   0.0000   0.0000   0.0000   0.0000  0.0000   
14     2018-08-15 00:00:00   0.0000   2.0000   0.0000   0.0000  0.0000   
15     2018-08-16 00:00:00   0.0000   0.0000   0.0000   0.0000  0.0000   
16     2018-08-17 00:00:00   0.0000   0.0000   0.0000   0.0000  0.0000   
17     2018-08-18 00:00:00   0.0000   0.0000   0.0000   0.0000  0.0000   
18     2018-08-19 00:00:00   0.0000   0.0000   0.0000   0.0000  0.0000   
19     2018-08-20 00:00:00   0.0000   0.0000   0.0000   0.0000  0.0000   
20     2018-08-21 00:00:00   0.0000   0.0000   0.0000   0.0000  0.0000   
21     2018-08-22 00:00:00   2.0000   0.0000   0.0000   0.0000  0.0000   
22     2018-08-23 00:00:00   0.0000   0.0000   4.0000   0.0000  0.0000   
23     2018-08-24 00:00:00   0.0000   0.0000   8.0000   0.0000  0.0000   
24     2018-08-25 00:00:00   0.0000   0.0000   0.0000   0.0000  0.0000   
25     2018-08-26 00:00:00   0.0000   0.0000   0.0000   6.0000  0.0000   
26     2018-08-27 00:00:00   0.0000   0.0000   0.0000   0.0000  0.0000   
27     2018-08-28 00:00:00   0.0000   4.0000   0.0000   0.0000  0.0000   
28     2018-08-29 00:00:00   0.0000   0.0000   0.0000   0.0000  2.0000   
29     2018-08-30 00:00:00   0.0000   0.0000   0.0000   0.0000  0.0000   
30     2018-08-31 00:00:00   0.0000   0.0000   0.0000   0.0000  0.0000   
Total                  NaN  10.0001  11.0001  16.0001  10.0001  5.0001   

        emp_06  emp_07  emp_08   emp_09   ...     emp_23   emp_24  emp_25  \
0       0.0000  0.0000  0.0000   0.0000   ...     0.0000   0.0000  0.0000   
1       0.0000  0.0000  0.0000   0.0000   ...     4.0000   0.0000  0.0000   
2       8.0000  0.0000  0.0000   0.0000   ...     0.0000   0.0000  0.0000   
3       0.0000  0.0000  0.0000   0.0000   ...     0.0000   0.0000  5.0000   
4       0.0000  0.0000  0.0000   0.0000   ...     0.0000   0.0000  0.0000   
5       0.0000  0.0000  0.0000   0.0000   ...     0.0000   0.0000  0.0000   
6       0.0000  0.0000  0.0000   0.0000   ...     0.0000   0.0000  0.0000   
7       0.0000  0.0000  0.0000   0.0000   ...     0.0000   0.0000  0.0000   
8       0.0000  0.0000  0.0000   0.0000   ...     0.0000   0.0000  0.0000   
9       0.0000  0.0000  0.0000   0.0000   ...     0.0000   0.0000  0.0000   
10      0.0000  0.0000  0.0000   0.0000   ...     0.0000   5.0000  0.0000   
11      0.0000  0.0000  6.0000   3.0000   ...     0.0000   0.0000  0.0000   
12      0.0000  2.0000  0.0000   0.0000   ...     0.0000   0.0000  0.0000   
13      0.0000  0.0000  0.0000   0.0000   ...     0.0000   0.0000  0.0000   
14      0.0000  0.0000  0.0000   0.0000   ...     0.0000   0.0000  0.0000   
15      0.0000  0.0000  0.0000   0.0000   ...     0.0000   0.0000  0.0000   
16      0.0000  0.0000  0.0000   0.0000   ...     0.0000   0.0000  0.0000   
17      0.0000  5.0000  0.0000   0.0000   ...     0.0000   0.0000  0.0000   
18      0.0000  0.0000  3.0000   0.0000   ...     0.0000   6.0000  0.0000   
19      0.0000  0.0000  0.0000   0.0000

In [31]:
df_emp['emp_16']

0         0.0000
1         0.0000
2         0.0000
3         0.0000
4         0.0000
5         2.0000
6         0.0000
7         0.0000
8         0.0000
9         0.0000
10        0.0000
11        0.0000
12        0.0000
13        0.0000
14        0.0000
15        0.0000
16        0.0000
17        0.0000
18        0.0000
19        0.0000
20        0.0000
21        0.0000
22        0.0000
23        0.0000
24        0.0000
25        3.0000
26        0.0000
27        0.0000
28        0.0000
29        0.0000
30       12.0000
Total    13.0001
Name: emp_16, dtype: float64